In [2]:
!pip install -r C:/Users/ouchaou/Desktop/ML/requirements.txt

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna

# ----------------------------------------------------------------
# Étape 1 : Chargement des données et nettoyage
# ----------------------------------------------------------------
con = duckdb.connect("C:/Users/ouchaou/Desktop/ML/src/data/trades.duckdb")
query = "select * from trades_with_indicators_view where datetime > '2024-03-01 00:10:00' ;"
df = con.execute(query).fetchdf()
con.close()

# Suppression des lignes avec valeurs manquantes
df = df.dropna()
print(f"Nombre de valeurs NaN dans le DataFrame: {df.isna().sum().sum()}")

# ----------------------------------------------------------------
# Étape 2 : Création de la cible et sélection des features
# ----------------------------------------------------------------
df['target'] = df['close_price'].shift(-1)
df = df.dropna()  # Supprimer les lignes NaN après le shift

features = [
    'BB_upper', 'min_price', 'max_price', 'EMA_50', 'open_price'
]



X = df[features]
y = df['target']

# ----------------------------------------------------------------
# Étape 3 : Division train/test et normalisation
# ----------------------------------------------------------------
# Division des données en ensembles d'entraînement (80%) et de test (20%)
# test_size=0.2 signifie que 20% des données sont réservées pour le test
# shuffle=False conserve l'ordre chronologique des données, important pour les séries temporelles
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ----------------------------------------------------------------
# Étape 4 : Recherche d'hyperparamètres avec validation croisée temporelle
#-----------------------------------------------------------------
# def objective(trial):
#     # Nous fixons un nombre d'estimateurs élevé pour laisser la place à l'early stopping
#     n_estimators = trial.suggest_int("n_estimators", 100, 300)

#     # Réduction de l'espace de recherche pour concentrer la recherche sur des plages pertinentes
#     learning_rate = trial.suggest_float("learning_rate", 0.01, 0.1, log=True)  # Plage restreinte pour éviter un learning_rate trop élevé (>0.3)
#     max_depth = trial.suggest_int("max_depth", 3, 5)  # On explore une plage restreinte pour éviter des modèles trop complexes
#     subsample = trial.suggest_float("subsample", 0.7, 0.9)  # Plage affinée autour de 0.8

#     # Création du modèle avec early stopping activé
#     model = GradientBoostingRegressor(
#         n_estimators=n_estimators,
#         learning_rate=learning_rate,
#         max_depth=max_depth,
#         subsample=subsample,
#         random_state=42,
#         n_iter_no_change=10,         # Arrêt anticipé si aucune amélioration pendant 10 itérations
#         validation_fraction=0.1      # Utilisation de 10% des données d'entraînement pour l'early stopping
#     )

#     # Validation croisée temporelle (ici 5-fold) pour évaluer la performance de cette configuration
#     # Si le dataset est très volumineux, il peut être nécessaire de passer à 3-fold pour réduire le coût
#     tscv = TimeSeriesSplit(n_splits=5)
#     rmse_scores = []
#     for train_index, valid_index in tscv.split(X_train_scaled):
#         X_tr, X_val = X_train_scaled[train_index], X_train_scaled[valid_index]
#         y_tr, y_val = y_train.iloc[train_index], y_train.iloc[valid_index]
        
#         model.fit(X_tr, y_tr)
#         y_pred = model.predict(X_val)
#         rmse = np.sqrt(mean_squared_error(y_val, y_pred))
#         rmse_scores.append(rmse)
    
#     # Retourne la moyenne des RMSE obtenues sur les différentes splits
#     return np.mean(rmse_scores)

# # Création de l'étude Optuna en mode minimisation de l'erreur RMSE
# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=10)

# print("Meilleurs hyperparamètres trouvés :", study.best_trial.params)


# ----------------------------------------------------------------
# Étape 5 : Validation avec TimeSeriesSplit
# ----------------------------------------------------------------
tscv = TimeSeriesSplit(n_splits=10)
cv_rmse = []  # Stocker les erreurs pour chaque split

for train_index, val_index in tscv.split(X_train_scaled):
    X_tr, X_val = X_train_scaled[train_index], X_train_scaled[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    model = GradientBoostingRegressor(
        n_estimators=178,
        learning_rate=0.045,
        max_depth=3,
        subsample=0.85,
        random_state=42,
        n_iter_no_change=10,
        validation_fraction=0.1
    )

    model.fit(X_tr, y_tr)
    
    # Validation
    y_pred_val = model.predict(X_val)
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    cv_rmse.append(rmse_val)

# Afficher la moyenne des RMSE sur les folds
print(f"Validation RMSE moyenne sur TimeSeriesSplit : {np.mean(cv_rmse):.2f}")

# ----------------------------------------------------------------
# Étape 6 : Entraînement du modèle final avec toutes les données d'entraînement
# ----------------------------------------------------------------
best_model = GradientBoostingRegressor(
    n_estimators=178,
    learning_rate=0.045,
    max_depth=3,
    subsample=0.85,
    random_state=42,
    n_iter_no_change=10,
    validation_fraction=0.1
)

best_model.fit(X_train_scaled, y_train)

# ----------------------------------------------------------------
# Étape 7 : Évaluation finale sur test set
# ----------------------------------------------------------------
y_pred_train = best_model.predict(X_train_scaled)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f"RMSE sur données d'entraînement: {rmse_train}")

y_pred_test = best_model.predict(X_test_scaled)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f"RMSE sur données de test: {rmse_test}")

# 1) MAE, MAPE, R²
mae_test = mean_absolute_error(y_test, y_pred_test)

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_test = mean_absolute_percentage_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print(f"MAE sur données de test: {mae_test}")
print(f"MAPE sur données de test: {mape_test:.2f}%")
print(f"R² sur données de test: {r2_test}")

# ----------------------------------------------------------------
# Étape 8 : Visualisation des erreurs et prédictions
# ----------------------------------------------------------------
plt.figure(figsize=(12, 6))
plt.plot(y_test.reset_index(drop=True), label='Valeurs Réelles')
plt.plot(y_pred_test, label='Prédictions')
plt.legend()
plt.title("Comparaison entre valeurs réelles et prédictions")
plt.xlabel("Index")
plt.ylabel("Prix de clôture")
plt.show()

# Importance des features
importances = best_model.feature_importances_
feature_importance_df = pd.DataFrame({'feature': features, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)
print(feature_importance_df)

plt.figure(figsize=(12, 6))
plt.bar(feature_importance_df['feature'], feature_importance_df['importance'])
plt.xticks(rotation=90)
plt.title("Importance des caractéristiques")
plt.xlabel("Caractéristiques")
plt.ylabel("Importance")
plt.show()

# ----------------------------------------------------------------
# Étape 9 : Enregistrement du modèle final
# ----------------------------------------------------------------
joblib.dump(best_model, 'gradient_boosting_model.pkl')
print("Modèle enregistré sous 'gradient_boosting_model.pkl'")


Nombre de valeurs NaN dans le DataFrame: 0


In [44]:
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']